In [4]:
from sympy import *
from sympy.matrices import Matrix
from sympy.physics.mechanics import LagrangesMethod, Lagrangian
from sympy.physics.mechanics import ReferenceFrame, Particle, Point
from sympy.physics.mechanics import dynamicsymbols, kinetic_energy
from sympy.physics.mechanics import mprint, mlatex
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from sympy.plotting import plot

In [15]:
m,xg,yg,zg,Ix,Iy,Iz,Ixy, Ixz, Iyx, Iyz, Izx, Izy = symbols("m x_g y_g z_g I_x I_y I_z I_xy I_xz I_yx I_yz I_zx I_zy")
u,v,w,p,q,r = symbols("u v w p q r")

In [13]:
#u,v,w,p,q,r = dynamicsymbols("u v w p q r")
#u = dynamicsymbols("u")
#v = dynamicsymbols("v")
#w = dynamicsymbols("w")
#p = dynamicsymbols("p")
#q = dynamicsymbols("q")
#r = dynamicsymbols("r")

Rigid-Body System Inertia Matriz

In [30]:
#MRB = Matrix([[m, 0, 0, 0, m*zg -m*yg]])
MRB = Matrix([[m, 0, 0, 0, m*zg, -m*yg],[0, m, 0, -m*zg, 0, m*xg],[0, 0, m, -m*yg, -m*xg, 0],[0, -m*zg, m*yg, Ix, -Ixy, -Ixz],[m*zg, 0, -m*xg, -Iyx, Iy, -Iyz],[-m*yg, m*xg, 0, -Izx, -Izy, Iz]])
MRB

Matrix([
[     m,      0,      0,      0,  m*z_g, -m*y_g],
[     0,      m,      0, -m*z_g,      0,  m*x_g],
[     0,      0,      m, -m*y_g, -m*x_g,      0],
[     0, -m*z_g,  m*y_g,    I_x,  -I_xy,  -I_xz],
[ m*z_g,      0, -m*x_g,  -I_yx,    I_y,  -I_yz],
[-m*y_g,  m*x_g,      0,  -I_zx,  -I_zy,    I_z]])

In [24]:
#m,xg,yg,zg,Ix,Iy,Iz,Ixy, Ixz, Iyx, Iyz, Izx, Izy = 1, 0.5, 0.5, 0.5,1,1,1,0.5,0.5,0.5,0.5,0.5,0.5

In [41]:
#A = MRB._eval_simplify
#MRB.subs([(m,3.5),(yg,5)])
MRB_num = MRB.subs({m:1,xg:0.5,yg:0.5,zg:0.5,Ix:1,Iy:1,Iz:1,Ixy:0.5,Ixz:0.5,Iyx:0.5,Iyz:0.3,Izx:0.3,Izy:0.3})
MRB_num

Matrix([
[   1,    0,    0,    0,  0.5, -0.5],
[   0,    1,    0, -0.5,    0,  0.5],
[   0,    0,    1, -0.5, -0.5,    0],
[   0, -0.5,  0.5,    1, -0.5, -0.5],
[ 0.5,    0, -0.5, -0.5,    1, -0.3],
[-0.5,  0.5,    0, -0.3, -0.3,    1]])

Rigid-Body Coriolis and Centripetal Matrix

In [18]:
zero3x3 = Matrix.zeros(3)
zero3x3

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

In [19]:
centripetal = Matrix([[-m*(yg*q+zg*r), m*(yg*p+w),m*(zg*p-v)],[m*(xg*q-w), -m*(zg*r+xg*p),m*(zg*p+u)],[m*(xg*r+v), m*(yg*r-u),-m*(xg*p+yg*q)]])
centripetal

Matrix([
[-m*(q*y_g + r*z_g),      m*(p*y_g + w),      m*(p*z_g - v)],
[     m*(q*x_g - w), -m*(p*x_g + r*z_g),      m*(p*z_g + u)],
[     m*(r*x_g + v),      m*(r*y_g - u), -m*(p*x_g + q*y_g)]])

In [20]:
inertia = Matrix(([0, -Iyz*q-Ixz*p+Iz*r,Iyz*r+Ixy*p-Iy*q],[Iyz*q+Ixz*p-Iz*r, 0, -Ixz*r-Ixy*q+Ix*p],[-Iyz*r-Ixy*p+Iy*q, Ixz*r+Ixy*q-Ix*p,0]))
inertia

Matrix([
[                       0, -I_xz*p - I_yz*q + I_z*r, I_xy*p - I_y*q + I_yz*r],
[ I_xz*p + I_yz*q - I_z*r,                        0, I_x*p - I_xy*q - I_xz*r],
[-I_xy*p + I_y*q - I_yz*r, -I_x*p + I_xy*q + I_xz*r,                       0]])

In [23]:
CRB = Matrix([[zero3x3,centripetal],[-centripetal,inertia]])
CRB

Matrix([
[                0,                 0,                 0,       -m*(q*y_g + r*z_g),            m*(p*y_g + w),           m*(p*z_g - v)],
[                0,                 0,                 0,            m*(q*x_g - w),       -m*(p*x_g + r*z_g),           m*(p*z_g + u)],
[                0,                 0,                 0,            m*(r*x_g + v),            m*(r*y_g - u),      -m*(p*x_g + q*y_g)],
[m*(q*y_g + r*z_g),    -m*(p*y_g + w),    -m*(p*z_g - v),                        0, -I_xz*p - I_yz*q + I_z*r, I_xy*p - I_y*q + I_yz*r],
[   -m*(q*x_g - w), m*(p*x_g + r*z_g),    -m*(p*z_g + u),  I_xz*p + I_yz*q - I_z*r,                        0, I_x*p - I_xy*q - I_xz*r],
[   -m*(r*x_g + v),    -m*(r*y_g - u), m*(p*x_g + q*y_g), -I_xy*p + I_y*q - I_yz*r, -I_x*p + I_xy*q + I_xz*r,                       0]])

In [25]:
#Test BlockMatrix
#PRB = BlockMatrix([[zero3x3,centripetal],[-centripetal,inertia]])
#PRB

In [27]:
CRB_num = CRB.subs({m:1,xg:0.5,yg:0.5,zg:0.5,Ix:1,Iy:1,Iz:1,Ixy:0.5,Ixz:0.5,Iyx:0.5,Iyz:0.3,Izx:0.3,Izy:0.3,u:1,v:0,w:0.5,p:0,q:0,r:0})
CRB_num

Matrix([
[  0,    0,  0,    0, 0.5, 0],
[  0,    0,  0, -0.5,   0, 1],
[  0,    0,  0,    0,  -1, 0],
[  0, -0.5,  0,    0,   0, 0],
[0.5,    0, -1,    0,   0, 0],
[  0,    1,  0,    0,   0, 0]])

In [29]:
V = Matrix([[1],[0],[0],[0],[0],[0]])
TRB = CRB_num * V
TRB

Matrix([
[  0],
[  0],
[  0],
[  0],
[0.5],
[  0]])